In [1]:
import glob
path = '/Users/taniajogesh/Documents/CCNPL/TestData/*.txt' 
files = glob.glob(path)
textlist = []
for i in xrange( 0, len(files)):
    textlist.append(open(files[i], 'r').read())
filenames = glob.glob(path)
filenames = [w.replace('/Users/taniajogesh/Documents/CCNPL/TestData/', '') for w in filenames]
names = [w.replace('.txt', '') for w in filenames]

In [2]:
import re
urlList = []
r = re.compile("https?:\/\/*")
for i in xrange( 0, len(textlist)):
    urlList.append([x for x in textlist[i].split("\n") if r.match(x)])

In [3]:
urlNames = zip(names,urlList)

In [4]:
#urlNamesTupple[5]

In [5]:
cleanurl =[]
for name, urllist in urlNames:
    for url in urllist:
        cleanurl.append((name,url))

In [6]:
cleanurl[300]

('April_2017',
 'http://www.financialexpress.com/economy/india-marks-dramatic-change-renewable-energy-capacity-addition-beats-conventional-in-fy17/618605/')

In [7]:
AllUrls=[]
for name,url in cleanurl:
    new_url = url.replace(' ','')
    AllUrls.append((name,new_url))

In [8]:
from newspaper import Article

def html_to_text( urls ):
    a = Article(urls) 
    a.download()
    a.parse()
    #a.nlp()
    text = a.text
    title = a.title
    authors = a.authors
    date = a.publish_date
    source = a.source_url
    #keywords = a.keywords
    data =(text,title,authors,date, source)
    return(data)

In [9]:
import pandas as pd
testURLs = pd.DataFrame(AllUrls)
testURLs.columns = ["month_yr","url"]
testURLs.head()

,month_yr,url
0,April_2011,http://www.thehindu.com/sci-tech/agriculture/a...
1,April_2011,http://www.thehindu.com/todays-paper/tp-nation...
2,April_2011,http://timesofindia.indiatimes.com/city/surat/...
3,April_2011,http://www.financialexpress.com/news/Green-Ind...
4,April_2011,http://timesofindia.indiatimes.com/city/surat/...


In [12]:
URLcount=testURLs.groupby('month_yr').agg('count')
URLcount.to_csv("URLCount")

In [13]:
import os
import pickle

if os.path.exists("Test_data_pickle.p"):
    print("Test data already downloaded")
    Testdata = pickle.load(open("Test_data_pickle.p", "rb"))
else:
    Testdata =[]
    Narticles = testURLs["url"].size
    print "Extracting data from online news weblinks...\n"
    for i in xrange(0, Narticles):
        try:
            Testdata.append((html_to_text(testURLs["url"][i]),i))
            if( (i+1)%100 == 0 ):
                print "Downloading %d of %d\n" % ( i+1, Narticles )
        except:
            pass
    pickle.dump(Testdata, open("Test_data_pickle.p", "wb"))    

Test data already downloaded


In [14]:
testdf = pd.DataFrame(eval('Testdata'))
testdf.columns=['data','ind'] #data column is a list of text title authors and data

In [15]:
#use map to separate list into individual columns
testdf["text"]=testdf.data.map(lambda x: x[0] if x !=[] else None)
testdf["title"]=testdf.data.map(lambda x: x[1] if x !=[] else None)
testdf["authors"]=testdf.data.map(lambda x: x[2] if x !=[] else None)
testdf["date"]=testdf.data.map(lambda x: x[3] if x !=[] else None)
testdf["source"]=testdf.data.map(lambda x: x[4] if x !=[] else None)

In [16]:
testdf.drop('data', inplace=True, axis=1)

In [17]:
testURLs.reset_index(inplace=True)
testdf =pd.merge(testdf, testURLs, left_on='ind', right_on='index', how='outer')

In [18]:
testdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6290 entries, 0 to 6289
Data columns (total 9 columns):
ind         6278 non-null float64
text        6278 non-null object
title       6278 non-null object
authors     6278 non-null object
date        2614 non-null object
source      6278 non-null object
index       6290 non-null int64
month_yr    6290 non-null object
url         6290 non-null object
dtypes: float64(1), int64(1), object(7)
memory usage: 491.4+ KB


In [69]:
print(testdf.head())

   ind                                               text  \
0  0.0  more-in\n\n“Potentially catastrophic” impacts ...   
1  1.0  An Assistant Professor of GITAM University Ins...   
2  2.0  SURAT: Over 200 delegates from across the coun...   
3  3.0  Climate change is at the top of government age...   
4  4.0  SURAT: The city resilience strategy after a st...   

                                               title  \
0  Prepare for long-term climate change impacts o...   
1                         Research on climate change   
2        Experts to discuss sustainable urban living   
3                                    Green India Inc   
4        Resilience strategy for city ready: Experts   

                                             authors  date  \
0  [Project Will Be On Display At Iit-delhi S Ope...  None   
1                                                 []  None   
2                                                 []  None   
3                                     [Michael B